In [120]:
#|default_exp datasets

In [121]:
#|export
import torch, torchvision, itertools, numpy as np, random
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import v2
import torch.nn.functional as F
from pathlib import Path
from itertools import islice

In [122]:
torch.cuda.is_available()

True

In [123]:
file_path = '../kin6-mini'

In [124]:
#| export
def get_video_paths(file_path):
    """
    Retrieves all videos from path and separates each subdirectory to its own class label

    Args:
        file_path (string): string path of desired directory

    Returns:
        video_paths [str]: list of string paths to video data.
        labels [int]: list of labels as integers
    """
    p = Path(file_path)
    video_paths = []
    labels = []
    sub_dirs = [dir for dir in p.iterdir() if dir.is_dir()]
    for i,sub_dir in enumerate(sub_dirs):
        paths = [str(p) for p in sub_dir.iterdir()]
        labels.extend([i for _ in range(len(paths))])
        video_paths.extend(paths)
    return video_paths, labels

In [125]:
video_paths, labels = get_video_paths('../kin6/train')

In [126]:
len(video_paths), len(labels)

(3000, 3000)

`get_frame_indices` currently drops remaining part of video by default if there are less frames remaining than clip_length.

Can be configured to return clips of less than determined clip length.

In [127]:
#| export
swin_transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((256,), interpolation=v2.functional.InterpolationMode.BILINEAR),
    v2.CenterCrop(224),
    v2.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [128]:
#| export
def format_clip(clip):
    """
    Formats clip based on Pytorch SwinTranformer3D inference example 
    https://pytorch.org/vision/0.18/models/generated/torchvision.models.video.swin3d_s.html#torchvision.models.video.swin3d_s

    Args:
        clip (tensor): clip of TCHW dimensions

    Returns:
        formatted_clip (tensor): dimensions CTHW
    """
    formatted_frames = []
    # Transforming clips of dimensions TCHW
    
    for frame in torch.unbind(clip):
        formatted_frames.append(swin_transforms(frame))
    # Swin3D transformer requires channels first CTHW
    return torch.stack(formatted_frames).permute(1,0,2,3)

Create CustomDataset that handles transformations and pass onto DataLoader

In [129]:
#| export
def get_clip(video_path, clip_length=32):
    """
    Reads video and return random batch of clip_length

    Args:
        video_path (str): path to video file
        clip_length (int): length of image clip

    Returns:
        clip (tensor): clip of captured frames (TCHW)
    """
    frames, _, _ = torchvision.io.read_video(video_path, output_format='TCHW')
    total_frames = len(frames)
    if total_frames <= 32:
        indices = torch.arange(0, total_frames)
        clip = frames[indices]
        # short clip is padded with zeros to achieve 32 frame length tensor
        pad = (0, 0, 0, 0, 0, 0, 0, 32-clip.shape[0])
        clip = F.pad(clip, pad, value=0)
        
    else:
        random_start = int(np.random.randint(0, total_frames-clip_length))
        indices = torch.arange(random_start, random_start+clip_length)
        clip = frames[indices]
    
    
    return clip

In [130]:
clip = torch.randn((17,3,224,224))
pad = (0, 0, 0, 0, 0, 0, 0, 32-clip.shape[0])
clip = F.pad(clip, pad, value=0)

In [131]:
clip.shape

torch.Size([32, 3, 224, 224])

In [132]:
clip = torch.randn((1,3,224,224))
clip.shape

torch.Size([1, 3, 224, 224])

In [133]:
pad = (0, 0, 0, 0, 0, 0, 0, 1)
clip = F.pad(clip, pad, value=0)

In [134]:
clip.shape

torch.Size([2, 3, 224, 224])

In [135]:
int(np.random.randint(0, 300-32))

134

In [136]:
clip = get_clip(video_paths[0], clip_length=32)
clip.shape

torch.Size([32, 3, 256, 454])

In [137]:
res = format_clip(clip)
res.shape, res.dtype

(torch.Size([3, 32, 224, 224]), torch.float32)

In [138]:
#| export
class CustomVideoDataset(Dataset):
    def __init__(self, video_paths, labels, clip_length=32, drop_last=True):
        self.video_paths, self.labels, self.clip_length, self.drop_last = video_paths, labels, clip_length, drop_last

    def __len__(self): return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        clip = get_clip(video_path, self.clip_length)
        label = torch.tensor(self.labels[idx])
        clip = format_clip(clip)
        return clip, label

In [139]:
train_ds = CustomVideoDataset(video_paths, labels)

In [140]:
x, y = train_ds[0]

In [141]:
x.shape, y

(torch.Size([3, 32, 224, 224]), tensor(0))

In [142]:
x[0].dtype

torch.float32

In [143]:
len(train_ds)

3000

Creating the custom DataLoader

Function handling the whole data processing

In [144]:
video_paths, labels = get_video_paths('../kin6/test')
test_ds = CustomVideoDataset(video_paths, labels)
test_dl = DataLoader(test_ds, 4, shuffle=True)

In [145]:
len(video_paths)

292

In [146]:
len(test_dl)

73

In [147]:
#| export 
def create_datasets(train_path, test_path, clip_length):
    """
    Creates training and test datasets.

    Args:
        train_path (string): path to training data directory
        test_path (string): path to testing data directory
        clip_length (int): number of frames per clip
        
    Returns:
        train_ds (Dataset): Pytorch dataset for training
        test_ds (Dataset): Pytorch dataset for testing
    """
    if not clip_length: clip_length=32
    train_paths, train_labels = get_video_paths(train_path)
    test_paths, test_labels = get_video_paths(test_path)
    train_ds, test_ds = CustomVideoDataset(train_paths, train_labels, clip_length), CustomVideoDataset(test_paths, test_labels, clip_length)
    return train_ds, test_ds

In [148]:
#| export
def create_dataloaders(train_path, test_path, clip_length, bs):
    """
    Creates dataloaders from trainining and testing

    Args:
        train_path (string): path to train data
        test_path (string): path to test data
        clip_length (int): length of each clip taken from video
        bs (int): batch size value

    Returns: 
        train dataloader (DataLoader)
        test dataloader (DataLoader)
    """
    if not clip_length: clip_length=32
    if not bs: bs = 4
    train_paths, train_labels = get_video_paths(train_path)
    test_paths, test_labels = get_video_paths(test_path)
    train_ds, test_ds = CustomVideoDataset(train_paths, train_labels, clip_length), CustomVideoDataset(test_paths, test_labels, clip_length)
    train_dl, test_dl = DataLoader(train_ds, batch_size=bs, shuffle=True), DataLoader(test_ds, batch_size=bs)
    return train_dl, test_dl

In [149]:
train_dl, test_dl = create_dataloaders('../kin6/train', '../kin6/test', clip_length=32, bs=4)

In [150]:
3000 // 4, 292 // 4

(750, 73)

In [151]:
len(train_dl), len(test_dl)

(750, 73)

In [152]:
xb, yb = next(iter(train_dl))
xb.shape

torch.Size([4, 3, 32, 224, 224])

In [153]:
yb.shape

torch.Size([4])

In [154]:
xb, yb = next(iter(test_dl))
xb.shape, xb.dtype

(torch.Size([4, 3, 32, 224, 224]), torch.float32)

In [155]:
#| export
def prepare_sim_data(train_path: str, test_path: str, clip_length: int, num_partitions: int, batch_size: int, val_ratio: float = 0.1):
    """This function partitions the training set into N disjoint
    subsets, each will become the local dataset of a client. This
    function also subsequently partitions each traininset partition
    into train and validation. The test set is left intact and will
    be used by the central server to asses the performance of the
    global model.

    Args:
        train_path (str): Path to train data
        test_path (str): Path to test data
        clip_length (int): Number of frames per video clip
        num_partitions (int): Number of partitions to create from original data
        batch_size (int): DataLoader batch size
        val_ratio (float): % of train data reserved for validation

    Returns:
        list of trainloaders [DataLoader]
        list of validation loaders [DataLoader]
        testloader (DataLoader)
    """

    # get datasets
    trainset, testset = create_datasets(train_path, test_path, clip_length)

    # split trainset into `num_partitions` trainsets
    num_images = len(trainset) // num_partitions

    partition_len = [num_images] * num_partitions

    trainsets = random_split(
        trainset, partition_len, torch.Generator().manual_seed(2023)
    )

    # create dataloaders with train+val support
    trainloaders = []
    valloaders = []
    for trainset_ in trainsets:
        num_total = len(trainset_)
        num_val = int(val_ratio * num_total)
        num_train = num_total - num_val

        for_train, for_val = random_split(
            trainset_, [num_train, num_val], torch.Generator().manual_seed(2023)
        )

        trainloaders.append(
            DataLoader(for_train, batch_size=batch_size, shuffle=True, num_workers=2)
        )
        valloaders.append(
            DataLoader(for_val, batch_size=batch_size, shuffle=False, num_workers=2)
        )

    # create dataloader for the test set
    testloader = DataLoader(testset, batch_size=batch_size)

    return trainloaders, valloaders, testloader

In [156]:
trainloaders, valloaders, testloader = prepare_sim_data('../kin6/train', '../kin6/test', 32, 30, 4)

In [157]:
train_partition = trainloaders[0].dataset

In [158]:
partition_indices = train_partition.indices
print(f'number of videos: {len(partition_indices)}')

number of videos: 90


# Export

In [159]:
import nbdev; nbdev.nbdev_export()